# Tree Based Methods
## Classification and Regression Trees (CART)

Decision trees, whether they are used for a continuous (regression) or discrete (classification) target variable, implement an algorithm called *Recursive Binary Splitting*.

### Recursive Binary Splitting

1. Search over the entire feature space, looking for the feature ($j$) and the value within that feature ($v_j$) that, when used as the split feature and value, results in the maximum reduction in the specified loss function. This will result in two subsets of the data.

2. Repeat step 1 on each of the two subsets until some stopping criteria is reached (usually a significantly "pure" subset is created, referred to as a "leaf" node).

##### Regression

With each successive split of a tree, the predicted value for all observations that fall into that subset (or region in the feature space if one is thinking about things geometrically) is the mean of the training observations that are in that subset/region. This will be a constant, mathematically defined as:

$$
c_m = \frac{\sum_{i \in R_m}^n y_i }{\left| R_m \right|}
$$

Translation: "The predicted value ($c_m$) for any observation that falls into the subset/region $m$ will be the average of the target variable for all training observations that fall into the same subset/region $m$." 

The loss function that is most commonly used to identify the "best" split feature $j$ and value $v_j$ is the RSS (Residual Sum of Squares).

$$
min \left( \sum_m^R min \left( \sum_{x_i \in R_m}^n (y_i - c_m)^2 \right) \right)
$$

Since each split is a binary split and there will only be two subsets, the above equation can be expanded to:

$$
min \left( min \left[ \sum_{x_i \in R_{m1}}^n (y_i - c_{m1})^2 \right] + min \left[ \sum_{x_i \in R_{m2}}^n (y_i - c_{m2})^2 \right] \right)
$$

Where $c_{m1}$ and $c_{m2}$ are the constants in regions $R_1$ and $R_2$.

**An important thing to note is that when using a regression tree to model a continuous relationship, the target variable will be discretized (even if it is a rather fine/precise discretization)**

##### Classification

In the classification setting, the algorithm seeks to make its two child nodes as pure as possible, where a completely pure node would be one with only one class. If the target variable can take on any one of a possible $K$ classes, then the predicted value for all observations that fall into the $m^{th}$ node will be the majority class in that node, (i.e. the class with the highest proportion in in node $m$). This can be written as:

$$
\left( \hat{y_i} | x_i \in R_m \right) = max \left( P_{mk} \right)
$$

which can be read as: "The predicted value for observation $i$ given that observation $i$ is in region/node $R_m$ is equal to the the class with highest proportion in that region $P_{mk}$." $P_{mk}$ is defined as:

$$
P_{mk} = \frac{1}{N_m} \sum_{x_i \in R_m} I(y_i = k)
$$

Translation: "The proportion of observations that belong to the $k^{th}$ class that are in region/node $m$ ($P_{mk}$) is equal to the number of observations that belong to class $k$ ($I(y_i = k)$) divided by the total number of observations in node m ($N_m$)"

The abstract structure of the loss function that a classification tree seeks to minimize is:

$$
loss = \frac{n_1}{N_m} G_1 + \frac{n_2}{N_m} G_2
$$

where:
* $N_m$ is the number of observations in node $m$ (i.e. the parent node that will be split)
* $n_1$ and $n_2$ are the number of observations in the "left" and "right" daughter nodes that will result from the split, respectively.
* $G_1$ and $G_2$ are the impurity measures of the "left" and "right" daughter nodes, respectively. The metric used to measure impurity can vary (see below).

As one can see, each impurity measure is weighted by the number of observations in its node, which allows the daughter node with more observations to be weighted more appropriately when considering a split.

###### Impurity Measures

There are 4 impurity measures that one could use to measure the impurity of a child node: 

1. Misclassification Error
2. Gini Index
3. Gini Impurity
4. Cross-Entropy

Since Gini Impurity and Cross-Entropy are used most frequently in modern day implementations of this algorithm, I will focus on these two. *All of these impurity measure will take on a low value (close to 0) when the $m^{th}$ node is significantly pure.*

* **Gini Impurity**:
    
$$
G_m = 1 - \sum_{k=1}^K P_{mk}^2
$$

Translation: "The gini impurity of node $m$ is equal to one minus the sum of the squared proportions of each class in node $m$"

* **Cross-Entropy**:
    
$$
E_m = - \sum_{k=1}^K P_{mk} log_2 P_{mk}
$$

Translation: "The Cross-Entropy in node $m$ ($E_m$) is equal to the summation across all $k$ classes in node $m$ of the product of the proportion of the $k^{th}$ class ($P_{mk}$) and the log of the proportion of the $k^{th}$ class ($log P_{mk}$), all multiplied by negative one."

* **Gini Index**:

    * Similar to Entropy in measurement of node purity, however the Gini Index measures the probability of misclassifying a single observation if it was randomly labeled according to the distribution of the classes in the sample.

$$G = \sum_i^K p_{mk}(1~-~p_{mk})$$ where $p_{mk}$ is the proportion of class $k$ in the $m$th node.

* Information Gain:

    * Note that in the below equation, $H(S)$ and $H(C_i)$ can be **either** the Entropy of the parent ($S$) and child ($C_i$) or the Gini Index of the parent ($S$) and child ($C_i$).

$$IG(S, C) = H(S)~-~\sum_{C_i\epsilon C} \frac{| C_i|}{|S|}H(C_i)$$

"The Information Gain of a split given the sample space and the 'children' (divisions of the sample space) of the split ($IG(S, C)$) is equal to the entropy/gini of the total sample space ($H(S)$) minus the summation over all splits (children) of the sample space of the size of the child node divided by the size of the sample space (parent node) ($\frac{|C_i|}{|S|}$) multiplied by the entropy/gini of the child node (division of sample space.)"

In [ ]:
import numpy as np
from collections import Counter

def discrete_entropy(y):
    '''
    INPUT:
        - y: 1d numpy array
    OUTPUT:
        - float

    Return the entropy of the array y.
    '''
    counter = Counter(y)
    total = len(y)
    ent = 0
    for i in counter.keys():
        ent += counter[i]/total * np.log2(counter[i] / total)
    return -1 * ent

def continuous_entropy(y):
    '''
    INPUT:
        - y: 1d numpy array
    OUTPUT:
        - float

    Return the entropy of the array y.
    '''
    ent = 0
    for x in np.arange(y.shape[0]):
        ent += (y[x] - y.mean())**2 * np.log2((y[x] - y.mean())**2)
    return -1 * ent

def gini(y):
    '''
    INPUT:
        - y: 1d numpy array
    OUTPUT:
        - float

    Return the gini impurity of the array y.
    '''

    counter = Counter(y)
    total = len(y)
    gini = 0
    for i in counter.keys():
        gini += counter[i]/total * (1 - (counter[i] / total))
    return gini

def information_gain(y, y1, y2, criteria='entropy'):
    '''
    Calculates the Information Gain from a binary split of y into y1
    and y2.

    INPUT:
        - y: 1d numpy array
        - y1: 1d numpy array (labels for subset 1)
        - y2: 1d numpy array (labels for subset 2)
    OUTPUT:
        - float

    Return the information gain of making the given split.
    '''
    counter = Counter(y)
    children = [y1, y2]
    summation_term = 0
    if criteria == 'entropy':
        for x, i in enumerate(counter.keys()):
            summation_term += counter[i] / len(y) * discrete_entropy(children[x])
        return discrete_entropy(y) - summation_term
    elif criteria == "gini":
        for x, i in enumerate(counter.keys()):
            summation_term += counter[i] / len(y) * gini(children[x])
        return gini(y) - summation_term


def continous_info_gain(parent_array, child_array_1, child_array_2):
    parent_var = np.var(parent_array)
    child_1_var = np.var(child_array_1)
    child_2_var = np.var(child_array_2)
    return parent_var - (len(child_array_1)/len(parent_array) * child_1_var) - (len(child_array_2)/len(parent_array) * child_2_var)

```